In [11]:
import os
import cv2
import joblib
import numpy as np
import mediapipe as mp
import time
import keyboard
import pygame
from gtts import gTTS

In [14]:
# Initialize Pygame for sound effects
pygame.mixer.init()

# Load the model from the same directory as the input file
model_path = os.path.join("D:/dataset/viscom/TA", "asl_alphabet_CatBoostClassifier.pkl")
model = joblib.load(model_path)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

DRAW_HANDS = True

# Map predicted labels to corresponding text
label_to_text = {
    'A': 'A',
    'B': 'B',
    'C': 'C',
    'D': 'D',
    'E': 'E',
    'F': 'F',
    'G': 'G',
    'H': 'H',
    'I': 'I',
    'J': 'J',
    'K': 'K',
    'L': 'L',
    'M': 'M',
    'N': 'N',
    'O': 'O',
    'P': 'P',
    'Q': 'Q',
    'R': 'R',
    'S': 'S',
    'T': 'T',
    'U': 'U',
    'V': 'V',
    'W': 'W',
    'X': 'X',
    'Y': 'Y',
    'Z': 'Z',
    '0.0': '0',
    '1.0': '1',
    '2.0': '2',
    '3.0': '3',
    '4.0': '4',
    '5.0': '5',
    '6.0': '6',
    '7.0': '7',
    '8.0': '8',
    '9.0': '9',
}

def get_landmarks(landmarks, is_right):
    landmark_tensor = []

    for i in range(21):
        landmark_tensor += [
            landmarks[i].x if is_right else 1 - landmarks[0].x,
            landmarks[i].y,
            landmarks[i].z
        ]

    return np.array(landmark_tensor).reshape(1, -1)

def convert_prediction_to_text(prediction):
    # Use the label to text mapping
    return label_to_text.get(prediction[0][0], '')

# Initialize variables for character accumulation
current_word = ""
current_sentence = ""
selected_label = ""  # New variable for the selected label
last_keypress_time = 0
keypress_delay = 0.5  # Set the delay in seconds
key_pressed = False  # Flag to track whether the key has been pressed for the current gesture

# Capture video from the default camera (index 0)
cap = cv2.VideoCapture(0)

# Use the Hands module from MediaPipe
with mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # Apply horizontal flip for a mirrored effect
        image = cv2.flip(image, 1)

        # Convert the image to RGB format
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process the image with the Hands module
        results = hands.process(image)

        # Convert the image back to BGR format
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If hands are detected in the image
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks and hand connections on the image
                if DRAW_HANDS:
                    mp.solutions.drawing_utils.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                        mp.solutions.drawing_styles.get_default_hand_connections_style())

                # Check if the detected hand is the right hand
                is_right = results.multi_handedness[0].classification[0].label == 'Right'

                # Get the prediction for the hand landmarks
                prediction = model.predict(get_landmarks(hand_landmarks.landmark, is_right))

                # If a valid prediction is obtained
                if len(prediction) == 1 and len(prediction[0]) == 1:
                    # Convert the prediction to text
                    text = convert_prediction_to_text(prediction)

                    # Update the current word and sentence
                    current_word += text
                    current_sentence += text

                    # Draw the text on the image
                    cv2.putText(image, text, (45, 375), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (255, 0, 0), 10)  # Warna biru untuk teks
                    cv2.putText(image, text, (45, 375), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 2, (255, 255, 255), 5)  # Warna putih untuk teks

                    # Check if the key has been pressed and update the selected label
                    if text not in selected_label and not key_pressed:
                        selected_label += text
                        key_pressed = True
                        # Tambahkan logika pemutaran suara di sini
                        pygame.mixer.music.load("sound_effect.mp3")
                        pygame.mixer.music.play()

        # Display the selected label on the image
        cv2.putText(image, selected_label, (45, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 10)
        cv2.putText(image, selected_label, (45, 450), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5)


        # Display the image with the drawn annotations
        cv2.imshow('MediaPipe Hands', image)

        # Check for key events
        key = cv2.waitKey(1)
        if key == ord('q'):
            cap.release()
            cv2.destroyWindow('MediaPipe Hands')
            break
        elif key == 13:  # Check for the 'Enter' key (key code 13)
            key_pressed = False  # Reset the flag when the 'Enter' key is pressed
            current_word += " "  # Add a space to separate words in the current sentence
        elif key == 32:  # Check for the 'Space' key (key code 32)
            current_sentence += selected_label + " "  # Add the selected label to the current sentence
            selected_label = ""  # Reset the selected label

# Release the video capture object
cap.release()

# Display the accumulated word and sentence
# print("Current Word:", current_word)
print("Current Sentence:", current_sentence)
print("Selected Label:", selected_label)

Current Sentence: ZZZZZZZZZZZCCCCCCEEEEEEEEESSSSASSSSASSASSSAAASAAAASASSSSASAAASAASAAASSSSSSSSSSSSESEESSEESSSSSSSSSSEEEEESSEESEEEEEEJJJZJJEEZJEEEJJJZZEJJZJZZZJJSZZZSZNSSSSSSSAEEEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAASASEEEESEESSESSSSSEEEEEEESSSESEEESSSSSSXZZZZZZZZJZZJZJZZZZZZZZZZZZZZRZZ0NNNNNN0N00NXNNNNN00NNNNNNNNNNNNNNNNNNNNNNNNNNN00XX0NNNN000NX600000XZZNZNNXZ6XZZ666Z6ZZ6XZXZ66ZXXXZXXNNNNNNNNZ6Z66MJ66Z6Z6ZZZNZ44RRRRRRBBBBBBBBBBBBBBBBBBBBBBBBBBBBEEEESSAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANANAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAZAZZZZAAZZZZABN AZZZZAAZZZZZSSSSEEEESASAAAAAAAAAAAAAAAASAAAAA AAAAAAAAAAASAAASSS AAAAAAAAAASASSSSSSSSSSSAAASSSSAASAASSSASAANNNNNNNNNNNNNNNSAAAAAAAAAAAAAAAAAAAAAAAAAASSSEEEEEEEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSEEFEB55555LLLLLLLZLZEEECCCCC000000000000000000000000000000000000ZRRRRRRRRRRRRRRRRRRURRUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUASU UUUUUUUUUURRRRR
Selected Label: 
